In [ ]:
!pip install -q qiskit qiskit-nature qiskit-aer pyscf qiskit_algorithms

In [ ]:
# pylint: disable=line-too-long
import qiskit_nature
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_algorithms.optimizers import SLSQP,SPSA
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

qiskit_nature.settings.use_pauli_sum_op = False  # pylint: disable=undefined-variable
# pylint: enable=line-too-long
from qiskit_nature.second_q.drivers import PySCFDriver
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2
import numpy as np
import pyscf


from qiskit_nature.second_q.algorithms import ExcitedStatesEigensolver
from qiskit_algorithms import NumPyEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, QEOM, EvaluationRule
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper

import os

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from qiskit.primitives import Estimator

def get_problem(nameAtoms, coordAtoms):

    molecule = MoleculeInfo(
        symbols=nameAtoms,
        coords=coordAtoms,
        multiplicity=1,
        charge=0,
    )
    driver = PySCFDriver.from_molecule(molecule)

    properties = driver.run()
    # print('particles: ',properties.num_particles)
    # print('orbitals:', properties.num_spatial_orbitals)

    problem =  ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=2).transform(properties)

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals


    mapper = ParityMapper(problem.num_particles)
    return num_particles, num_spatial_orbitals, problem, mapper

In [ ]:
def read_coords(fileName):
    with open(fileName, 'r') as infile:
        nameAtoms = []
        for line in infile :
            if line.startswith('HETATM') or line.startswith('ATOM'):
                    try:
                        nameAtoms.append(str.strip(line[12:14]))
                        coordAtoms = np.vstack([coordAtoms, [float(line[30:38]), float(line[38:46]), float(line[46:54])]])
                    except NameError:
                        coordAtoms = np.array([[line[30:38],line[38:46], line[46:54]]], float)

        return nameAtoms, coordAtoms

In [ ]:

def distance(coord1, coord2):
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

def adjust_coords(coords, symbols, percentage_change):
    new_coords = np.array(coords)
    num_symbols = len(symbols)

    distances = np.zeros((num_symbols, num_symbols))
    for i in range(num_symbols):
        for j in range(i+1, num_symbols):
            distances[i, j] = distance(coords[i], coords[j])
            distances[j, i] = distances[i, j]

    adjusted_distances = distances * (1 + percentage_change)

    for i in range(num_symbols):
        for j in range(i+1, num_symbols):
            diff_vector = np.array(coords[i]) - np.array(coords[j])
            current_distance = np.linalg.norm(diff_vector)
            scale_factor = adjusted_distances[i, j] / current_distance
            new_coords[i] = coords[i] - diff_vector * (1 - scale_factor) / 2
            new_coords[j] = coords[j] + diff_vector * (1 - scale_factor) / 2

    return np.round(new_coords, 3).tolist()

In [ ]:
import datetime
import json

directory_files = sorted(os.listdir('/content/drive/MyDrive/pdb'))
print(directory_files)

numpy_solver = NumPyMinimumEigensolver()
noiseless_estimator = Estimator()
optimizer = SLSQP(maxiter=10)
percentages = [0.10,0.15,0.00,-0.10,-0.15]



with open('/content/drive/MyDrive/pdbOutput/output'+datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S') + '.txt', 'w') as writefile:
    writefile.write('{"results":[\n')
    for file in directory_files:
      if file.endswith('.pdb'):
          result = {}
          (nameAtoms,coordAtoms) = read_coords('/content/drive/MyDrive/pdb/'+file)
          result['file'] = file
          result['nameAtoms'] = nameAtoms
          result['calculations']= []
          print(file)
          # print("atoms: ", nameAtoms)
          for percentage in percentages:
              calculation={}
              # print('Diff from original: ',percentage)
              if percentage != 0.00:
                coords = adjust_coords(coordAtoms,nameAtoms,percentage)
              else :
                coords = np.array(coordAtoms).tolist()
              # print(coords)
              (num_particles, num_spatial_orbitals, problem, mapper) = get_problem(nameAtoms,coords)

              init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
              var_form = UCCSD(
                  num_spatial_orbitals, num_particles, mapper, initial_state=init_state
              )
              vqe = VQE(
                  noiseless_estimator,
                  var_form,
                  optimizer,
                  initial_point=[0.0] * var_form.num_parameters,
              )
              gse = GroundStateEigensolver(mapper, vqe)

              solved = gse.solve(problem)
              calculation['diff'] = percentage
              calculation['coords'] = coords
              calculation['energy'] = solved.total_energies[0]

              result['calculations'].append(calculation)
              # print('energy: ', solved.total_energies)
              # print('-------------------')

          # print(result)
          energies = [calc["energy"] for calc in result['calculations']]

          min_energy = min(energies)
          # print(min_energy)
          # print(result['calculations'][3]['energy'])

          result['isBaseCorrect'] = bool(result['calculations'][2]['energy'] == min_energy)
          writefile.write(json.dumps(result))
          writefile.write(',\n')
          # print('-----------------------------------------------')
    writefile.write(']}')




['acetald.pdb', 'asf3.pdb', 'b2h6.pdb', 'bcl3.pdb', 'beh2.pdb', 'bf3.pdb', 'bh3.pdb', 'brf3.pdb', 'c2h2.pdb', 'c2h3n.pdb', 'c2h4.pdb', 'c2h6.pdb', 'caffeine.pdb', 'ccln.pdb', 'cf2ccl2.pdb', 'cf4.pdb', 'ch2cl2.pdb', 'ch5n.pdb', 'cl2o.pdb', 'clf3.pdb', 'co2.pdb', 'cos.pdb', 'cs2.pdb', 'ethanol.pdb', 'etoh.pdb', 'form.pdb', 'geh4.pdb', 'h2.pdb', 'h2s.pdb', 'h2se.pdb', 'h3po4.pdb', 'hbr.pdb', 'hcl.pdb', 'hcn.pdb', 'hn3.pdb', 'hno2.pdb', 'hno3.pdb', 'i2.pdb', 'icl.pdb', 'n2.pdb', 'nocl.pdb', 'p4s3.pdb', 'pcl5.pdb', 'propanol.pdb', 'sbh3.pdb', 'seocl2.pdb', 'snh4.pdb', 'ticl4.pdb']
acetald.pdb
asf3.pdb
b2h6.pdb
bcl3.pdb
beh2.pdb
bf3.pdb
bh3.pdb
brf3.pdb
c2h2.pdb
c2h3n.pdb
c2h4.pdb
c2h6.pdb
caffeine.pdb


KeyboardInterrupt: 